# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

In [3]:
len(CORD19_CSV['doi'])

77448

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

In [5]:
#delete Nans
doi_counted = doi.value_counts()
doi_counted

10.31729/jnma.5498               2
10.1016/j.dsx.2020.04.012        2
10.1016/j.febslet.2012.01.005    1
10.1007/978-3-642-15742-4_31     1
10.1136/medhum-2020-011969       1
                                ..
10.1007/978-3-030-51466-2_11     1
10.7717/peerj.9733               1
10.1093/cid/ciaa677              1
10.3389/fmed.2020.00384          1
10.1096/fj.202000805             1
Name: doi, Length: 74302, dtype: int64

In [6]:
def AffiliationsFromScopusByDOI(client, doi):
    """obtain addiotional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

In [7]:
## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()
## Initialize client
client = ElsClient(config['apikey'])
# https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4

In [8]:
df_current_extra_info = pd.read_pickle('extra_info_CS5099.pkl')
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
41413,"[{'affiliation-city': 'Perth', 'affilname': 'M...","{'srctype': 'j', 'prism:issueIdentifier': '11'..."
41414,"[{'affiliation-city': 'Guangzhou', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85096874998', ..."
41415,"[{'affiliation-city': 'Sydney', 'affilname': '...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."
41416,"[{'affiliation-city': 'Hangzhou', 'affilname':...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."


In [9]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

41418

In [10]:
%%time
i = len_df_current_extra_info
dict_new_extra_info = dict()
len_dois = len(doi_counted)
while i < len_dois:
    print("Position: " + str(i) + " -> " +  doi_counted.index[i])
    dict_new_extra_info[i] = AffiliationsFromScopusByDOI(client, doi_counted.index[i])
    i = i + 1 

Position: 41418 -> 10.1016/j.scitotenv.2020.144549
Position: 41419 -> 10.1101/2020.02.16.20023754
Position: 41420 -> 10.1007/s15010-011-0206-5
Position: 41421 -> 10.1007/bf03405419
Position: 41422 -> 10.1111/j.1750-2659.2012.00365.x
Position: 41423 -> 10.1091/mbc.e20-06-0374
Position: 41424 -> 10.21203/rs.3.rs-34729/v1
Position: 41425 -> 10.3389/fped.2020.584694
Position: 41426 -> 10.1016/j.jairtraman.2020.101917
Position: 41427 -> 10.1016/j.aimed.2020.07.007
Position: 41428 -> 10.1007/s13222-016-0245-2
Position: 41429 -> 10.1101/2020.08.03.20167189
Position: 41430 -> 10.1016/j.jnma.2020.08.010
Position: 41431 -> 10.1097/md.0000000000022022
Position: 41432 -> 10.1186/s12909-020-02181-7
Position: 41433 -> 10.1186/s13054-015-1083-6
Position: 41434 -> 10.1186/s12864-017-3801-8
Position: 41435 -> 10.1371/journal.pone.0204605
Position: 41436 -> 10.1101/2020.09.09.20191296
Position: 41437 -> 10.1111/jgs.16752
Position: 41438 -> 10.1016/j.ajem.2021.01.058
Position: 41439 -> 10.1007/s12015-020

Position: 41599 -> 10.1016/j.antiviral.2017.02.007
Position: 41600 -> 10.1101/2020.04.25.20077396
Position: 41601 -> 10.1007/s00112-013-2984-6
Position: 41602 -> 10.1016/j.biopha.2020.110738
Position: 41603 -> 10.1093/nar/gky932
Position: 41604 -> 10.1016/j.psychres.2020.113130
Position: 41605 -> 10.1093/ajhp/zxaa391
Position: 41606 -> 10.1186/s12917-020-02341-3
Position: 41607 -> 10.3390/ijms21155469
Position: 41608 -> 10.1371/journal.pntd.0005693
Position: 41609 -> 10.1007/s10620-006-9480-y
Position: 41610 -> 10.1101/2020.05.19.20106914
Position: 41611 -> 10.1016/j.jretconser.2020.102186
Position: 41612 -> 10.1111/j.1469-0691.2010.03413.x
Position: 41613 -> 10.3390/biom10091239
Position: 41614 -> 10.1371/journal.pone.0032157
Position: 41615 -> 10.1007/s11845-012-0833-6
Position: 41616 -> 10.1038/s41598-020-77170-3
Position: 41617 -> 10.1057/978-1-137-53675-4_4
Position: 41618 -> 10.1016/j.jinf.2020.05.052
Position: 41619 -> 10.1007/s10461-019-02480-1
Position: 41620 -> 10.1016/j.nmni

Position: 41780 -> 10.1134/s0026893319030154
Position: 41781 -> 10.1016/j.childyouth.2020.105231
Position: 41782 -> 10.1038/sj.mt.6300116
Position: 41783 -> 10.1016/j.jviromet.2011.04.007
Position: 41784 -> 10.1038/s41433-019-0716-4
Position: 41785 -> 10.14366/usg.20088
Position: 41786 -> 10.3390/diseases8040045
Position: 41787 -> 10.1155/2011/395690
Position: 41788 -> 10.1016/j.jpsychores.2020.110286
Position: 41789 -> 10.1093/nar/gkw1065
Position: 41790 -> 10.3390/nu12113406
Position: 41791 -> 10.1007/s12603-020-1366-8
Position: 41792 -> 10.1016/j.ajic.2020.06.206
Position: 41793 -> 10.1016/j.ijid.2020.09.1434
Position: 41794 -> 10.1016/j.jtcme.2020.12.002
Position: 41795 -> 10.1016/j.scs.2020.102468
Position: 41796 -> 10.1186/1471-2334-11-65
Position: 41797 -> 10.1016/j.hpopen.2020.100025
Position: 41798 -> 10.1101/2020.04.13.20063248
Position: 41799 -> 10.1186/s12859-020-03935-2
Position: 41800 -> 10.1007/s13149-011-0179-8
Position: 41801 -> 10.1101/2020.04.07.20057224
Position: 41

Position: 41964 -> 10.1007/s13365-014-0235-9
Position: 41965 -> 10.2147/idr.s270543
Position: 41966 -> 10.1016/j.cmi.2020.11.035
Position: 41967 -> 10.1371/journal.pgen.1000849
Position: 41968 -> 10.1016/j.jneuroim.2004.04.018
Position: 41969 -> 10.1016/j.idm.2020.08.012
Position: 41970 -> 10.1007/978-3-030-53288-8_23
Position: 41971 -> 10.1016/j.antiviral.2016.08.022
Position: 41972 -> 10.1371/journal.pone.0025697
Position: 41973 -> 10.1007/s42495-020-00044-z
Position: 41974 -> 10.3732/ajb.1400434
Position: 41975 -> 10.1634/theoncologist.2020-0516
Position: 41976 -> 10.1007/s12028-020-00974-8
Position: 41977 -> 10.4161/idp.27454
Position: 41978 -> 10.1016/j.dsx.2020.09.002
Position: 41979 -> 10.3934/publichealth.2020039
Position: 41980 -> 10.1093/jamia/ocaa210
Position: 41981 -> 10.1002/ame2.12083
Position: 41982 -> 10.1016/j.plabm.2020.e00188
Position: 41983 -> 10.1016/j.isjp.2020.07.005
Position: 41984 -> 10.1016/j.trsl.2011.12.009
Position: 41985 -> 10.1016/j.jmig.2020.08.486
Posit

Position: 42145 -> 10.1371/journal.pone.0007162
Position: 42146 -> 10.1007/s10989-020-10038-2
Position: 42147 -> 10.1093/ije/dyaa209
Position: 42148 -> 10.3389/fimmu.2020.592370
Position: 42149 -> 10.1101/2020.10.28.20221143
Position: 42150 -> 10.1084/jem.20192291
Position: 42151 -> 10.1371/journal.pone.0007837
Position: 42152 -> 10.1101/2020.06.21.20128181
Position: 42153 -> 10.1101/2020.11.20.391318
Position: 42154 -> 10.3201/eid1002.030759
Position: 42155 -> 10.1101/2020.06.12.20129650
Position: 42156 -> 10.1007/s00059-020-05001-2
Position: 42157 -> 10.1080/07391102.2020.1796804
Position: 42158 -> 10.1007/978-3-030-50420-5_25
Position: 42159 -> 10.1093/jpids/piaa123
Position: 42160 -> 10.1016/j.jcv.2009.09.030
Position: 42161 -> 10.1016/j.jinf.2010.06.016
Position: 42162 -> 10.3390/ijerph18020526
Position: 42163 -> 10.1371/journal.pone.0125447
Position: 42164 -> 10.1016/j.ijporl.2016.09.006
Position: 42165 -> 10.1101/2020.09.17.300335
Position: 42166 -> 10.3389/fimmu.2020.606333
Pos

...

Read document failed.
Position: 49714 -> 10.1007/s00705-018-3855-1
Read document failed.
Position: 49715 -> 10.3390/healthcare8040432
Read document failed.
Position: 49716 -> 10.1101/2021.01.07.21249366
Position: 49717 -> 10.1007/s40822-020-00162-1
Read document failed.
Position: 49718 -> 10.1016/j.ajp.2020.102119
Read document failed.
Position: 49719 -> 10.7759/cureus.12274
Position: 49720 -> 10.1016/j.chaos.2020.109846
Read document failed.
Position: 49721 -> 10.1101/2020.10.30.20223115
Position: 49722 -> 10.1007/s41603-020-00126-y
Read document failed.
Position: 49723 -> 10.1007/s10096-008-0495-5
Read document failed.
Position: 49724 -> 10.3390/ph14010017
Position: 49725 -> 10.3390/ijerph18020674
Read document failed.
Position: 49726 -> 10.2147/rmhp.s257473
Read document failed.
Position: 49727 -> 10.1111/j.1751-0813.1989.tb09716.x
Read document failed.
Position: 49728 -> 10.1111/puar.13251
Read document failed.
Position: 49729 -> 10.1097/cm9.0000000000000675
Read document failed.
P

Read document failed.
Position: 49844 -> 10.1007/s11948-007-9043-4
Read document failed.
Position: 49845 -> 10.1101/2020.04.19.048710
Position: 49846 -> 10.1016/j.jinf.2020.12.024
Read document failed.
Position: 49847 -> 10.1038/ng1698
Read document failed.
Position: 49848 -> 10.3390/cancers12113450
Read document failed.
Position: 49849 -> 10.1101/2020.10.22.20217604
Position: 49850 -> 10.3389/fpsyt.2020.00866
Read document failed.
Position: 49851 -> 10.1007/s10758-020-09454-6
Read document failed.
Position: 49852 -> 10.1016/j.rinp.2020.103811
Read document failed.
Position: 49853 -> 10.1136/bmjopen-2020-041295
Read document failed.
Position: 49854 -> 10.1128/mbio.01930-20
Read document failed.
Position: 49855 -> 10.1093/eurjpc/zwaa053
Read document failed.
Position: 49856 -> 10.3390/ijerph17145146
Read document failed.
Position: 49857 -> 10.1051/vetres/2009065
Read document failed.
Position: 49858 -> 10.1101/2020.09.12.294538
Position: 49859 -> 10.1038/s41598-020-79484-8
Read document

Position: 49975 -> 10.1016/j.mehy.2020.109898
Read document failed.
Position: 49976 -> 10.1080/07391102.2020.1768149
Read document failed.
Position: 49977 -> 10.3201/eid1801.110725
Read document failed.
Position: 49978 -> 10.3389/fmicb.2020.593857
Read document failed.
Position: 49979 -> 10.1016/s0140-6736(20)32466-1
Read document failed.
Position: 49980 -> 10.1016/j.remle.2020.05.006
Position: 49981 -> 10.1101/2020.03.17.20037457
Position: 49982 -> 10.1016/j.vaccine.2011.03.005
Read document failed.
Position: 49983 -> 10.1242/bio.046342
Read document failed.
Position: 49984 -> 10.3390/ijerph17239055
Read document failed.
Position: 49985 -> 10.1128/jcm.00875-20
Read document failed.
Position: 49986 -> 10.1016/j.jbvi.2020.e00181
Read document failed.
Position: 49987 -> 10.1016/j.cyto.2018.01.025
Read document failed.
Position: 49988 -> 10.1186/s13063-020-04728-x
Read document failed.
Position: 49989 -> 10.1101/2020.11.02.20215657
Position: 49990 -> 10.1101/2020.05.31.126615
Position: 49

Read document failed.
Position: 50107 -> 10.1101/2020.05.12.088542
Position: 50108 -> 10.1016/j.procs.2020.09.337
Read document failed.
Position: 50109 -> 10.1186/s13568-020-00991-y
Read document failed.
Position: 50110 -> 10.1186/1471-2105-15-s4-i1
Read document failed.
Position: 50111 -> 10.1186/s13031-020-00324-7
Read document failed.
Position: 50112 -> 10.1186/s40249-018-0435-2
Read document failed.
Position: 50113 -> 10.1007/978-3-030-50743-5_23
Read document failed.
Position: 50114 -> 10.1093/cid/ciaa684
Read document failed.
Position: 50115 -> 10.1016/j.amjsurg.2003.12.072
Read document failed.
Position: 50116 -> 10.1016/j.jbef.2020.100454
Read document failed.
Position: 50117 -> 10.1101/2020.09.08.20190769
Position: 50118 -> 10.1371/journal.pone.0243704
Read document failed.
Position: 50119 -> 10.1016/j.hrtlng.2020.10.012
Read document failed.
Position: 50120 -> 10.1016/j.jeurea.2021.01.001
Read document failed.
Position: 50121 -> 10.1007/s00520-020-05573-z
Read document failed

KeyboardInterrupt: 

In [22]:
# dict_new_extra_info_saver = dict()
# i = 41418
# while i < 47683:
#     #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
#     dict_new_extra_info_saver[i] = dict_new_extra_info[i]
#     i = i + 1 

# dict_new_extra_info = dict_new_extra_info_saver

In [23]:
dict_new_extra_info

{41418: {'affiliation': [{'affiliation-city': 'Sapporo',
    'affilname': 'Hokkaido University',
    'affiliation-country': 'Japan'},
   {'affiliation-city': 'Matsuyama',
    'affilname': 'Center for Marine Environmental Studies, Ehime University',
    'affiliation-country': 'Japan'}],
  'coredata': {'srctype': 'j',
   'eid': '2-s2.0-85099500778',
   'pubmed-id': '33477053',
   'prism:coverDate': '2021-05-15',
   'prism:aggregationType': 'Journal',
   'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85099500778',
   'subtypeDescription': 'Article',
   'dc:creator': {'author': [{'ce:given-name': 'Fuminari',
      'preferred-name': {'ce:given-name': 'Fuminari',
       'ce:initials': 'F.',
       'ce:surname': 'Miura',
       'ce:indexed-name': 'Miura F.'},
      '@seq': '1',
      'ce:initials': 'F.',
      '@_fa': 'true',
      'affiliation': {'@id': '60091846',
       '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60091846'},
      'ce:surname': 

In [24]:
df_new_extra_info = pd.DataFrame(dict_new_extra_info)

In [25]:
df_new_extra_info

,41418,41419,41420,41421,41422,41423,41424,41425,41426,41427,...,47673,47674,47675,47676,47677,47678,47679,47680,47681,47682
affiliation,"[{'affiliation-city': 'Sapporo', 'affilname': ...",None,"[{'affiliation-city': 'Regensburg', 'affilname...","{'affiliation-city': 'Ottawa', 'affilname': 'U...","[{'affiliation-city': 'Kisumu', 'affilname': '...","{'affiliation-city': 'Los Angeles', 'affilname...",None,"{'affiliation-city': 'Abha', 'affilname': 'Kin...","[{'affiliation-city': 'Beni Mellal', 'affilnam...","[{'affiliation-city': 'Toronto', 'affilname': ...",...,None,"[{'affiliation-city': 'Manchester', 'affilname...",None,None,"[{'affiliation-city': 'Xining', 'affilname': '...","[{'affiliation-city': 'Palakkad', 'affilname':...","{'affiliation-city': 'New Haven', 'affilname':...","[{'affiliation-city': 'Wuhan', 'affilname': 'T...","[{'affiliation-city': 'Porto', 'affilname': 'U...","[{'affiliation-city': 'Honolulu', 'affilname':..."
coredata,"{'srctype': 'j', 'eid': '2-s2.0-85099500778', ...",None,"{'srctype': 'j', 'prism:issueIdentifier': '2',...","{'srctype': 'j', 'eid': '2-s2.0-36048932302', ...","{'srctype': 'j', 'prism:issueIdentifier': '2',...","{'srctype': 'j', 'eid': '2-s2.0-85090990376', ...",None,"{'srctype': 'j', 'eid': '2-s2.0-85097624588', ...","{'srctype': 'j', 'eid': '2-s2.0-85090713875', ...","{'srctype': 'j', 'eid': '2-s2.0-85089528302', ...",...,None,"{'srctype': 'j', 'eid': '2-s2.0-85088800392', ...",None,None,"{'srctype': 'j', 'eid': '2-s2.0-85089982527', ...","{'srctype': 'j', 'eid': '2-s2.0-85088813502', ...","{'srctype': 'j', 'prism:issueIdentifier': '4',...","{'srctype': 'j', 'eid': '2-s2.0-85093096338', ...","{'srctype': 'j', 'eid': '2-s2.0-52149090972', ...","{'srctype': 'j', 'eid': '2-s2.0-85099427473', ..."


In [26]:
df_new_extra_transposed = df_new_extra_info.T
df_new_extra_transposed

,affiliation,coredata
41418,"[{'affiliation-city': 'Sapporo', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85099500778', ..."
41419,None,None
41420,"[{'affiliation-city': 'Regensburg', 'affilname...","{'srctype': 'j', 'prism:issueIdentifier': '2',..."
41421,"{'affiliation-city': 'Ottawa', 'affilname': 'U...","{'srctype': 'j', 'eid': '2-s2.0-36048932302', ..."
41422,"[{'affiliation-city': 'Kisumu', 'affilname': '...","{'srctype': 'j', 'prism:issueIdentifier': '2',..."
...,...,...
47678,"[{'affiliation-city': 'Palakkad', 'affilname':...","{'srctype': 'j', 'eid': '2-s2.0-85088813502', ..."
47679,"{'affiliation-city': 'New Haven', 'affilname':...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."
47680,"[{'affiliation-city': 'Wuhan', 'affilname': 'T...","{'srctype': 'j', 'eid': '2-s2.0-85093096338', ..."
47681,"[{'affiliation-city': 'Porto', 'affilname': 'U...","{'srctype': 'j', 'eid': '2-s2.0-52149090972', ..."


In [27]:
df_combined_extra_info = pd.concat([df_current_extra_info, df_new_extra_transposed],ignore_index=True)

In [29]:
df_combined_extra_info.to_pickle('extra_info_CS5099.pkl')
df_combined_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
47678,"[{'affiliation-city': 'Palakkad', 'affilname':...","{'srctype': 'j', 'eid': '2-s2.0-85088813502', ..."
47679,"{'affiliation-city': 'New Haven', 'affilname':...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."
47680,"[{'affiliation-city': 'Wuhan', 'affilname': 'T...","{'srctype': 'j', 'eid': '2-s2.0-85093096338', ..."
47681,"[{'affiliation-city': 'Porto', 'affilname': 'U...","{'srctype': 'j', 'eid': '2-s2.0-52149090972', ..."


In [17]:
entry = AffiliationsFromScopusByDOI(client, '10.1086/605034')
entry

Read document failed.


In [18]:
print (json.dumps(entry, ensure_ascii=False, indent=4))

null


In [19]:
df_test = pd.DataFrame.from_dict(entry['affiliation'], orient='columns')
df_test

TypeError: 'NoneType' object is not subscriptable

In [ ]:
entry['coredata']

In [ ]:
## ScienceDirect (full-text) document example using DOI
doi_doc = FullDoc(doi = '10.1016/S1525-1578(10)60571-5')
if doi_doc.read(client):
    print ("doi_doc.title: ", doi_doc.title)
    doi_doc.write()   
else:
    print ("Read document failed.")
doi_doc